In [4]:
import sqlite3
import random

class ChessLadder:
    def __init__(self, db_path="chess_ladder0.db"):
        self.db_path = db_path
        self._create_table()
        self.matches = []

    def _create_table(self):
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute(
                '''
                CREATE TABLE IF NOT EXISTS players (
                    id INTEGER PRIMARY KEY,
                    name TEXT UNIQUE,
                    rating INTEGER
                )
                '''
            )

    def add_player(self, player):
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("INSERT OR IGNORE INTO players (name, rating) VALUES (?, 1000)", (player,))

    def get_rating(self, player):
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT rating FROM players WHERE name=?", (player,))
            result = cursor.fetchone()
            return result[0] if result else None

    def set_rating(self, player, rating):
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("UPDATE players SET rating=? WHERE name=?", (rating, player))

    def print_standings(self):
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT name, rating FROM players ORDER BY rating DESC")
            standings = cursor.fetchall()

        print("Current Standings:")
        for idx, (player, rating) in enumerate(standings, start=1):
            print(f"{idx}. {player} - Rating: {rating}")

    def challenge(self, player1, player2):
        rating1 = self.get_rating(player1)
        rating2 = self.get_rating(player2)

        if rating1 is None or rating2 is None:
            print("Invalid players.")
            return

        print(f"{player1} challenges {player2} to a match!")
        result = input(f"{player1}'s result (win/draw/loss): ").lower()

        if result == "win":
            self.set_rating(player1, rating1 + 10)
            self.set_rating(player2, rating2 - 10)
            print(f"{player1} wins! Ratings updated.")
        elif result == "draw":
            print("It's a draw. Ratings unchanged.")
        elif result == "loss":
            self.set_rating(player1, rating1 - 10)
            self.set_rating(player2, rating2 + 10)
            print(f"{player2} wins! Ratings updated.")
        else:
            print("Invalid result. Match not recorded.")

    def simulate_matches(self, num_matches):
        all_players = [player[0] for player in self._get_all_players()]
        for _ in range(num_matches):
            players = random.sample(all_players, 2)
            self.challenge(players[0], players[1])

    def _get_all_players(self):
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT name, rating FROM players")
            return cursor.fetchall()

if __name__ == "__main__":
    ladder = ChessLadder()

    # Add players
    ladder.add_player("Alice")
    ladder.add_player("Bob")
    ladder.add_player("Charlie")

    # Simulate matches
    ladder.simulate_matches(5)

    # Print standings
    ladder.print_standings()


Bob challenges Charlie to a match!
Invalid result. Match not recorded.
Alice challenges Charlie to a match!
Invalid result. Match not recorded.
Alice challenges Charlie to a match!
Invalid result. Match not recorded.
Bob challenges Alice to a match!
Invalid result. Match not recorded.
Bob challenges Charlie to a match!
Invalid result. Match not recorded.
Current Standings:
1. Alice - Rating: 1000
2. Bob - Rating: 1000
3. Charlie - Rating: 1000


In [3]:
!pip install sqlite3

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3
